In [1]:
# Data Loading and Descriptive Statistics
import pandas as pd
dataset = pd.read_csv("Chapter7_Sand Production.csv")
print(dataset.describe)

<bound method NDFrame.describe of     No   TVD   TT   COH     Qg        Qw   BHFP     DD  EOVS   SPF  Hperf  \
0    1   319  105  22.0   42.3  5672.000  133.2   27.8   651   4.0   14.0   
1    2  3182  105  21.9   51.2    68.000  140.4   16.6   642   4.0   16.0   
2    3  3366  100  24.7   66.9   157.000  156.2   18.9   601   4.0    6.0   
3    4  3647  100  29.6   80.6    85.000  153.8   57.8   670   8.0   20.0   
4    5  4548   85  53.2   48.0   886.000  209.1   58.9   823   4.0   18.0   
5    6  4088   85  39.5   72.7   116.000  147.0   44.0   781   2.0   17.0   
6    7  2100  115  10.8   28.5   724.000  160.1    8.9   300   4.0   15.5   
7    8  1930  132   9.7   27.5   695.000  175.5   11.2   245   4.0   11.5   
8    9  2139  112  11.1   36.8   280.000  185.5    6.1   283   4.0   10.5   
9   10  2380  110  13.0   23.0    42.000  113.0   47.4   413   6.0   11.0   
10  11  1122  150   5.7  108.0     0.000  107.0    8.0   115  12.0   10.5   
11  12  1340  130   6.6   51.0    52.000  

In [2]:
# Using algorithm "K-nearest neighbors (KNN)" to train the model that predicts which well will produce sand.

# Define input (x) and target(y)
x = dataset.iloc[:,1:11]
y = dataset.iloc[:,11].values

# Scale input data between 0 and 1 as the data attributes have different ranges.
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
xnorm = pd.DataFrame(data = sc.fit_transform(x))

# Partition data into test and train
import numpy as np
seed = 50
np.random.seed(seed)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xnorm, y, test_size = 0.3)

# Train binary classifier (k-neighbor) and prediction on train and test data
# set
from sklearn.neighbors import KNeighborsClassifier
KNC = KNeighborsClassifier(n_neighbors = 3, leaf_size = 3)
KNC.fit(x_train, y_train)
y_train_predict = KNC.predict(x_train)
y_test_predict = KNC.predict(x_test)

In [3]:
# Using evaluation metrics of Accuracy (ratio of correct prediction to total prediction)
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(y_test, y_test_predict)
print('Accuracy_test: %f' %accuracy_test)

Accuracy_test: 0.777778


In [4]:
# Using evaluation metrics of Precision (if dataset is highly imbalanced, use Precision instead)
from sklearn.metrics import precision_score
precision_test = precision_score(y_test, y_test_predict)
print('Precision_test: %f' %precision_test)


Precision_test: 0.750000


In [5]:
# Using evaluation metrics of Recall (identifies the ability of the model to predict POSITIVE values)
from sklearn.metrics import recall_score
recall_test = recall_score(y_test, y_test_predict)
print('Recall_test: %f' %recall_test)

Recall_test: 1.000000


In [6]:
# Using evaluation metrics of F1 Score (harmonic mean of Precision and Recall)
from sklearn.metrics import f1_score
f1_test = f1_score(y_test, y_test_predict)
print('F1 score_train: %f' %f1_test)

F1 score_train: 0.857143


In [7]:
# Summary of Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_predict))

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.75      1.00      0.86         6

    accuracy                           0.78         9
   macro avg       0.88      0.67      0.68         9
weighted avg       0.83      0.78      0.74         9



In [8]:
# Cross-validation for Classification
from sklearn.model_selection import cross_val_score
scores = cross_val_score(KNC, xnorm, y, cv = 5, scoring = 'accuracy')
print("Cross-validtion scoresL: {}". format(scores))
print("Average cross-valdation score: {}". format(scores.mean()))

Cross-validtion scoresL: [0.66666667 1.         1.         0.66666667 1.        ]
Average cross-valdation score: 0.8666666666666666


In [9]:
# Stratified K-fold cross-validation for Classification
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 100)
scoresSK = cross_val_score(KNC, xnorm, y, cv = skfold, scoring = 'accuracy')
print(" StratifiedKFold cross-validation scores: {}". format(scoresSK))
print(" Average StratifiedKFold cross-validation score: {}". format(scoresSK.mean()))

 StratifiedKFold cross-validation scores: [1.         0.83333333 0.83333333 0.83333333 1.        ]
 Average StratifiedKFold cross-validation score: 0.9


In [10]:
from sklearn.model_selection import ShuffleSplit
sh_sp = ShuffleSplit(test_size = .25, train_size = .75, n_splits = 5, random_state = 50)
scoresSP = cross_val_score(KNC, xnorm, y, cv = sh_sp)
print(" Cross-validation scores: {}". format(scoresSP))
print(" Mean accuracy: {}". format(scoresSP.mean()))

 Cross-validation scores: [0.75 1.   0.75 0.75 0.75]
 Mean accuracy: 0.8


In [11]:
# Initial grid search of hyperparameter
from sklearn.neighbors import KNeighborsClassifier

# Define k-nearest neighbor's combination of hyper-parameters
highest_score = 0
i = 0
for n in [2, 3, 4, 5]:
    for w in ['uniform', 'distance']:
        for l in [2, 3, 4]:
# Train a model with each combination of hyper-parameters and calculate the aacuracy
            KNCG = KNeighborsClassifier(n_neighbors = n, weights = w, leaf_size = l)
            scores = cross_val_score(KNCG, xnorm, y, cv = 5, scoring = 'accuracy')
            scoreg = scores.mean()
            print("iteration:", i, "score:", scoreg, "NN:", n, "weight:", w, "leaf size:", l)
            i += 1
            
            if scoreg >= highest_score:
                highest_score = scoreg
                best_parameters = {'n_neighbors':n, 'weights':w, 'leaf_size':l}
print(" Highest score:", (highest_score))
print(" Best parameters:", format(best_parameters))

iteration: 0 score: 0.8933333333333333 NN: 2 weight: uniform leaf size: 2
iteration: 1 score: 0.8933333333333333 NN: 2 weight: uniform leaf size: 3
iteration: 2 score: 0.8933333333333333 NN: 2 weight: uniform leaf size: 4
iteration: 3 score: 0.9 NN: 2 weight: distance leaf size: 2
iteration: 4 score: 0.9 NN: 2 weight: distance leaf size: 3
iteration: 5 score: 0.9 NN: 2 weight: distance leaf size: 4
iteration: 6 score: 0.8666666666666666 NN: 3 weight: uniform leaf size: 2
iteration: 7 score: 0.8666666666666666 NN: 3 weight: uniform leaf size: 3
iteration: 8 score: 0.8666666666666666 NN: 3 weight: uniform leaf size: 4
iteration: 9 score: 0.8666666666666666 NN: 3 weight: distance leaf size: 2
iteration: 10 score: 0.8666666666666666 NN: 3 weight: distance leaf size: 3
iteration: 11 score: 0.8666666666666666 NN: 3 weight: distance leaf size: 4
iteration: 12 score: 0.7533333333333333 NN: 4 weight: uniform leaf size: 2
iteration: 13 score: 0.7533333333333333 NN: 4 weight: uniform leaf size: 3

In [12]:
# (GridSearchCV) Grid search with cross validation
from sklearn.model_selection import GridSearchCV
Neighbor = [2, 3, 4, 5]
Weight = ['uniform', 'distance']
Leaf = [2, 3, 4]
hyperparameters = dict(n_neighbors = Neighbor, weights = Weight, leaf_size = Leaf)
KNN = KNeighborsClassifier()
gridsearch = GridSearchCV(KNN, hyperparameters, cv = 5)
Best_Model = gridsearch.fit(xnorm, y)
print('Best n_neighbors:', Best_Model.best_estimator_.get_params()['n_neighbors'])
print('Best weights:', Best_Model.best_estimator_.get_params()['weights'])
print('Best leaf_size:', Best_Model.best_estimator_.get_params()['leaf_size'])
B = Best_Model.predict(xnorm)

from sklearn.metrics import classification_report
print(classification_report(y, B))

Best n_neighbors: 2
Best weights: distance
Best leaf_size: 2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        21

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



In [ ]:
# (RandomizedSearchCV) to search wthin a random set of hyperparameter combinations.
from sklearn.model_selection import RandomizedSearchCV
Neighbor = [2, 3, 4, 5]
Weight = ['uniform', 'distance']
Leaf = [2, 3, 4]
hyperparameters = dict(n_neighbors = Neighbor, weights = Weight, leaf_size = Leaf)
KNN = KNeighborsClassifier()
gridsearch_randomized = RandomizedSearchCV(KNN, hyperparameters, n_iter = 12, random_state = 1, cv = 5)
Best_Model = gridsearch_randomized.fit(xnorm, y)
print('Best n_neighbors:', Best_Model.best_estimator_.get_params()['n_neighbors'])
print('Best weights:', Best_Model.best_estimator_.get_params()['weights'])
print('Best leaf_size:', Best_Model.best_estimator_.get_params()['leaf_size'])
BRG = Best_Model.predict(xnorm)

from sklearn.metrics import classification_report
print(classification_report(y, BRG))

Best n_neighbors: 2
Best weights: distance
Best leaf_size: 4
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        21

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29

